# VeriRegime - 知识蒸馏训练

将CNN Teacher（74.62%）的知识蒸馏到MLP Student  

**目标**:  
- Student准确率: ≥ 63% (Teacher的85%)  
- 理想准确率: ≥ 67% (Teacher的90%)  
- 参数量减少: ~85%


## 1. 导入依赖


In [1]:
import sys
sys.path.append('..')  # 添加项目根目录

import torch
import platform

# 导入训练模块
from train import CNNVolatility, MLPStudent, create_dataloaders, DistillationTrainer

print(f"PyTorch版本: {torch.__version__}")
print(f"系统: {platform.system()}")


PyTorch版本: 2.9.1
系统: Darwin


## 2. 配置参数


In [2]:
# 数据配置（与CNN训练一致）
FEATURE_COLS = ['ema_5', 'ema_10', 'ema_20', 'rsi', 'macd', 'volume_ma_5', 'volume_ma_10']
SEQ_LENGTH = 240  # 4小时 = 240分钟
BATCH_SIZE = 512

# 蒸馏配置
TEMPERATURE = 4.0  # 温度参数（越大，分布越平滑）
ALPHA = 0.7  # 蒸馏损失权重（0.7蒸馏 + 0.3硬标签）
EPOCHS = 30
LR = 1e-3
EARLY_STOP_PATIENCE = 8

# 路径配置
TRAIN_CSV = '../data/train.csv'
VAL_CSV = '../data/val.csv'
TEST_CSV = '../data/test.csv'
TEACHER_CHECKPOINT = '../results/checkpoints/best_model.pth'  # CNN Teacher

# 设备配置
if platform.system() == 'Darwin' and torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
    print("🍎 使用MPS (Apple Silicon)")
elif torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print("🚀 使用CUDA")
else:
    DEVICE = torch.device('cpu')
    print("💻 使用CPU")

print(f"\n蒸馏配置:")
print(f"  温度T: {TEMPERATURE}")
print(f"  蒸馏权重α: {ALPHA}")
print(f"  硬标签权重: {1-ALPHA}")


🍎 使用MPS (Apple Silicon)

蒸馏配置:
  温度T: 4.0
  蒸馏权重α: 0.7
  硬标签权重: 0.30000000000000004


## 3. 加载数据


In [3]:
print("加载数据...")

train_loader, val_loader, test_loader, class_weights, stats = create_dataloaders(
    train_csv=TRAIN_CSV,
    val_csv=VAL_CSV,
    test_csv=TEST_CSV,
    feature_cols=FEATURE_COLS,
    seq_length=SEQ_LENGTH,
    batch_size=BATCH_SIZE
)

print(f"\n数据统计:")
print(f"  训练集: {stats['train_size']:,} 样本")
print(f"  验证集: {stats['val_size']:,} 样本") 
print(f"  测试集: {stats['test_size']:,} 样本")


加载数据...

数据统计:
  训练集: 681,105 样本
  验证集: 145,763 样本
  测试集: 145,763 样本


## 4. 加载CNN Teacher


In [4]:
print("\n加载CNN Teacher...")

# 创建Teacher模型
teacher = CNNVolatility(input_channels=len(FEATURE_COLS), seq_length=SEQ_LENGTH)

# 加载预训练权重
checkpoint = torch.load(TEACHER_CHECKPOINT, map_location=DEVICE)
teacher.load_state_dict(checkpoint['model_state_dict'])
teacher.eval()  # 设为评估模式

teacher_params = sum(p.numel() for p in teacher.parameters())
print(f"✅ Teacher加载成功")
print(f"  参数量: {teacher_params:,}")
print(f"  最佳F1: {checkpoint['best_f1']:.4f}")
print(f"  训练轮次: {checkpoint['epoch']}")



加载CNN Teacher...
✅ Teacher加载成功
  参数量: 35,778
  最佳F1: 0.7369
  训练轮次: 4


## 5. 创建MLP Student


In [5]:
print("\n创建MLP Student...")

# 输入维度 = seq_length * num_features
input_dim = SEQ_LENGTH * len(FEATURE_COLS)

student = MLPStudent(
    input_dim=input_dim,
    hidden_dims=[256, 128, 64],
    dropout_rate=0.3
)

student_params = sum(p.numel() for p in student.parameters())

print(f"✅ Student创建成功")
print(f"  输入维度: {input_dim}")
print(f"  隐藏层: 256 → 128 → 64 → 2")
print(f"  参数量: {student_params:,}")
print(f"  压缩比: {teacher_params / student_params:.1f}x")
print(f"  参数减少: {(1 - student_params/teacher_params)*100:.1f}%")



创建MLP Student...
✅ Student创建成功
  输入维度: 1680
  隐藏层: 256 → 128 → 64 → 2
  参数量: 471,618
  压缩比: 0.1x
  参数减少: -1218.2%


## 6. 知识蒸馏训练


In [6]:
print("\n初始化蒸馏训练器...")

config = {
    'temperature': TEMPERATURE,
    'alpha': ALPHA,
    'lr': LR,
    'weight_decay': 1e-4
}

distill_trainer = DistillationTrainer(
    teacher_model=teacher,
    student_model=student,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    device=DEVICE,
    config=config
)

print("\n开始蒸馏训练...")
print("=" * 60)

best_f1 = distill_trainer.train(
    epochs=EPOCHS, 
    early_stop_patience=EARLY_STOP_PATIENCE
)



初始化蒸馏训练器...

开始蒸馏训练...

Epoch 1/30
------------------------------------------------------------


Validating: 100%|██████████| 285/285 [00:03<00:00, 89.63it/s] 


训练 - Loss: 0.2482 | Acc: 76.06%
验证 - Loss: 0.2188 | Acc: 72.06% | F1: 0.7108
✅ 最佳模型已保存 (F1: 0.7108)

Epoch 2/30
------------------------------------------------------------


Validating: 100%|██████████| 285/285 [00:02<00:00, 97.41it/s] 


训练 - Loss: 0.2154 | Acc: 77.21%
验证 - Loss: 0.2276 | Acc: 72.05% | F1: 0.7121
✅ 最佳模型已保存 (F1: 0.7121)

Epoch 3/30
------------------------------------------------------------


...

训练 - Loss: 0.1710 | Acc: 79.89%
验证 - Loss: 0.1972 | Acc: 72.54% | F1: 0.7220

Epoch 29/30
------------------------------------------------------------


Validating: 100%|██████████| 285/285 [00:03<00:00, 93.13it/s] 


训练 - Loss: 0.1708 | Acc: 79.94%
验证 - Loss: 0.2040 | Acc: 72.46% | F1: 0.7217

Epoch 30/30
------------------------------------------------------------


Validating: 100%|██████████| 285/285 [00:03<00:00, 88.43it/s] 

训练 - Loss: 0.1705 | Acc: 79.95%
验证 - Loss: 0.1976 | Acc: 72.75% | F1: 0.7244


## 7. 测试Student模型


In [7]:
print("\n测试MLP Student...")

test_acc, test_f1, preds, labels = distill_trainer.test()

print("\n" + "=" * 60)
print("MLP Student测试结果")
print("=" * 60)
print(f"准确率: {test_acc*100:.2f}%")
print(f"F1分数: {test_f1:.4f}")



测试MLP Student...


Testing: 100%|██████████| 285/285 [00:01<00:00, 196.16it/s]


MLP Student测试结果
准确率: 72.48%
F1分数: 0.7218


## 8. Teacher vs Student 性能对比


In [16]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

print("\n评估CNN Teacher性能...")

# 在测试集上评估Teacher
teacher.eval()
teacher_preds = []

with torch.no_grad():
    for features, _ in test_loader:
        features = features.to(DEVICE)
        output = teacher(features)
        batch_preds = torch.argmax(output['logits'], dim=1)  # ✅ 改名避免覆盖
        teacher_preds.extend(batch_preds.cpu().numpy())

teacher_acc = accuracy_score(labels, teacher_preds)
teacher_f1 = f1_score(labels, teacher_preds, average='weighted')

print("\n" + "=" * 60)
print("Teacher vs Student 性能对比")
print("=" * 60)
print(f"\n{'指标':<20} {'Teacher (CNN)':<15} {'Student (MLP)':<15} {'保留率':<10}")
print("-" * 60)
print(f"{'参数量':<20} {teacher_params:<15,} {student_params:<15,} {student_params/teacher_params*100:>8.1f}%")
print(f"{'准确率':<20} {teacher_acc*100:<15.2f} {test_acc*100:<15.2f} {test_acc/teacher_acc*100:>8.1f}%")
print(f"{'F1分数':<20} {teacher_f1:<15.4f} {test_f1:<15.4f} {test_f1/teacher_f1*100:>8.1f}%")

# 判断是否达标
performance_ratio = test_acc / teacher_acc

print("\n" + "=" * 60)
if performance_ratio >= 0.90:
    print("🎉 优秀！Student保留了Teacher 90%+的性能")
elif performance_ratio >= 0.85:
    print("✅ 达标！Student保留了Teacher 85%+的性能")
elif performance_ratio >= 0.80:
    print("📊 良好！Student保留了Teacher 80%+的性能")
else:
    print("⚠️ 需改进！性能损失较大，建议调整蒸馏参数")

print(f"\n参数压缩比: {teacher_params/student_params:.1f}x")
print(f"性能保留率: {performance_ratio*100:.1f}%")
print(f"蒸馏效率: {performance_ratio / (student_params/teacher_params):.2f}")



评估CNN Teacher性能...

Teacher vs Student 性能对比

指标                   Teacher (CNN)   Student (MLP)   保留率       
------------------------------------------------------------
参数量                  35,778          471,618           1318.2%
准确率                  74.62           72.48               97.1%
F1分数                 0.7463          0.7218              96.7%

🎉 优秀！Student保留了Teacher 90%+的性能

参数压缩比: 0.1x
性能保留率: 97.1%
蒸馏效率: 0.07


## 9. 详细分类报告


In [17]:
print("\n" + "=" * 60)
print("MLP Student 分类报告")
print("=" * 60)
print(classification_report(
    labels, preds,
    target_names=['LOW', 'HIGH'],
    digits=4
))

# 混淆矩阵
from sklearn.metrics import confusion_matrix
import pandas as pd

cm = confusion_matrix(labels, preds)
cm_df = pd.DataFrame(
    cm,
    index=['真实LOW', '真实HIGH'],
    columns=['预测LOW', '预测HIGH']
)

print("\n混淆矩阵:")
print(cm_df)



MLP Student 分类报告


ValueError: Found input variables with inconsistent numbers of samples: [145763, 355]

## 10. 保存对比结果


In [ ]:
import json
import os

# 保存对比结果
results = {
    'teacher': {
        'params': teacher_params,
        'accuracy': float(teacher_acc),
        'f1_score': float(teacher_f1)
    },
    'student': {
        'params': student_params,
        'accuracy': float(test_acc),
        'f1_score': float(test_f1)
    },
    'distillation': {
        'temperature': TEMPERATURE,
        'alpha': ALPHA,
        'compression_ratio': float(teacher_params / student_params),
        'performance_retention': float(performance_ratio)
    }
}

os.makedirs('../results/metrics', exist_ok=True)
with open('../results/metrics/distillation_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n✅ 结果已保存: results/metrics/distillation_results.json")
